In [639]:
import numpy as np
import matplotlib.pyplot as plt

In [640]:
from loaders import clean_text_for_pattern_letter
from temp_text_puh import template_str
# input_str = clean_text_for_pattern_letter(template_str, out_len=len(template_str))
input_str = clean_text_for_pattern_letter(template_str, out_len=5000)
print(len(input_str))
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
               'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '"', '\'', '1']
print(f'{letters=}')

len(letters)=32
5000
letters=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '"', "'", '1']


In [641]:
def predict_train(input_str,
                  letters,
                  max_history=4,
                  max_index: int = 4,):
    WIN: dict[str, dict[str, int]] = {}

    HistoryHW: list[dict[str, int]] = []
    HistoryW: list[dict[str, dict[str, dict[int, int]]]] = []

    for h_idx in range(max_history):
        HistoryHW.append({})
        HistoryW.append({})

    IDX: list = []

    for letter in letters:
        WIN.setdefault(letter, {})

        for idx in range(max_index):
            WIN[letter].setdefault(idx, 0)

    prev_indices = [0]

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0:
            WIN[inp][prev_indices[0]] += 1
            continue

        IDX.append(prev_indices[0])

        IdxO = {}

        for h_idx in range(max_history):
            prev_idx = inp_idx - 1 - h_idx
            if prev_idx < 0:
                break

            prev_inp = input_str[prev_idx]

            HW = HistoryHW[h_idx]
            W = HistoryW[h_idx]

            key_HW = prev_inp + str(prev_indices[h_idx]) + ':' + inp

            if HW.get(key_HW, -1) != -1:
                inp_has_index = HW[key_HW]

                # idx_strength = W[prev_inp + str(prev_indices[h_idx])][inp][inp_has_index]

                IdxO.setdefault(inp_has_index, 0)
                IdxO[inp_has_index] += 1

        new_index = 0
        if len(IdxO) == 0:
            new_index = min(WIN[inp], key=WIN[inp].get)
        else:
            new_index = max(IdxO, key=IdxO.get)
        WIN[inp][new_index] += 1

        for h_idx in range(max_history):
            prev_idx = inp_idx - 1 - h_idx
            if prev_idx < 0:
                break

            prev_inp = input_str[prev_idx]

            key_I = prev_inp + str(prev_indices[h_idx])
            HistoryW[h_idx].setdefault(key_I, {})
            HistoryW[h_idx][key_I].setdefault(inp, {})
            HistoryW[h_idx][key_I][inp].setdefault(new_index, 0)

            stats = HistoryW[h_idx][key_I][inp][new_index]
            if stats == 0:
                HistoryW[h_idx][key_I][inp][new_index] += 1
            else:
                HistoryW[h_idx][key_I][inp][new_index] += 1 / (32 * stats)

            key_HW = prev_inp + str(prev_indices[h_idx]) + ':' + inp

            HistoryHW[h_idx].setdefault(key_HW, new_index)

        # recurtion
        prev_indices.insert(0, new_index)
        if len(prev_indices) > max_history:
            prev_indices = prev_indices[:-1]
    
    IDX.append(new_index)

    return HistoryW, HistoryHW, IDX

max_index_L1 = 32
max_history_L1 = 12

HistoryW, HistoryHW, IDX = predict_train(input_str, letters, max_history=max_history_L1, max_index=max_index_L1)

print(f'{len(IDX)}')

for h_idx in range(max_history_L1):
    print(f'{h_idx=}')
    for wk, wv in HistoryW[h_idx].items():
        print(wk, wv)


5000
h_idx=0
c0 {'h': {0: 1}}
h0 {'a': {0: 1}}
a0 {'p': {0: 1}}
p0 {'t': {0: 1}}
t0 {'e': {0: 1}}
e0 {'r': {0: 1.03125}}
r0 {' ': {0: 1}, 'h': {4: 1}}
 0 {'i': {0: 1}, 'b': {7: 1}, 't': {19: 1}}
i0 {'n': {0: 1}}
n0 {' ': {1: 1}}
 1 {'w': {0: 1}, 'c': {5: 1}, 'o': {24: 1}}
w0 {'h': {1: 1}}
h1 {'i': {1: 1}, 'r': {8: 1}}
i1 {'c': {1: 1}}
c1 {'h': {2: 1}}
h2 {' ': {2: 1}}
 2 {'w': {1: 1}, 'r': {9: 1}, 'i': {12: 1}}
w1 {'e': {1: 1}}
e1 {' ': {3: 1}, 'r': {17: 1}}
 3 {'a': {1: 1.03125}, 'i': {14: 1}}
a1 {'r': {1: 1}, 'n': {12: 1}}
r1 {'e': {2: 1}, ' ': {2: 1}}
e2 {' ': {4: 1.03125}}
 4 {'i': {2: 1.03125}, 't': {9: 1}, 'h': {23: 1}}
i2 {'n': {1: 1}, 's': {9: 1}}
n1 {'t': {1: 1}}
t1 {'r': {2: 1}}
r2 {'o': {0: 1}}
o0 {'d': {0: 1}}
d0 {'u': {0: 1}}
u0 {'c': {2: 1}}
c2 {'e': {3: 1}}
e3 {'d': {1: 1}, 'a': {22: 1}}
d1 {' ': {5: 1}}
 5 {'t': {2: 1}, 'a': {9: 1}, 'h': {19: 1}, 'i': {5: 1}}
t2 {'o': {1: 1}}
o1 {' ': {6: 1}}
 6 {'w': {2: 1}, 'f': {1: 1.03125}, 'a': {20: 1}}
w2 {'i': {3: 1}}
i3 {'n': {2

In [642]:
def reverse_predict_train(input_str: list,
                          IDX: list,
                          max_history=4,):
    HistoryW: list[dict[str, dict[str, dict[int, int]]]] = []
    
    for h_idx in range(max_history):
        HistoryW.append({})

    prev_indices = [0]

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0:
            continue

        new_index = IDX[inp_idx]

        for h_idx in range(max_history):
            prev_idx = inp_idx - 1 - h_idx
            if prev_idx < 0:
                break

            prev_inp = input_str[prev_idx]

            key_I = prev_inp + str(prev_indices[h_idx])
            HistoryW[h_idx].setdefault(key_I, {})
            HistoryW[h_idx][key_I].setdefault(inp, {})
            HistoryW[h_idx][key_I][inp].setdefault(new_index, 0)

            stats = HistoryW[h_idx][key_I][inp][new_index]
            if stats == 0:
                HistoryW[h_idx][key_I][inp][new_index] += 1
            else:
                HistoryW[h_idx][key_I][inp][new_index] += 1 / (32 * stats)

        # recurtion
        prev_indices.insert(0, new_index)
        if len(prev_indices) > max_history:
            prev_indices = prev_indices[:-1]

    return HistoryW


RevHistoryW = reverse_predict_train(input_str[::-1],
                                    IDX[::-1],
                                    max_history=max_history_L1, )

print(f'{len(IDX)=}')

for h_idx in range(max_history_L1):
    print(f'{h_idx=}')
    for wk, wv in RevHistoryW[h_idx].items():
        print(wk, wv)


len(IDX)=5000
h_idx=0
u0 {'o': {6: 1}, 'd': {0: 1}}
o6 {'y': {2: 1.03125, 10: 1.3262469848333007, 8: 1.0615530303030303}, 's': {5: 1.4401483341933956, 22: 1, 17: 1.0615530303030303}, 't': {27: 1.3022500574996982, 16: 1.6054425857659549, 28: 1.03125, 29: 1, 6: 1}, 'o': {6: 1.4401483341933956, 29: 1.03125, 28: 1.03125, 17: 1}, ' ': {20: 1.7370729405694407, 19: 1.1475456027178208, 22: 1.0615530303030303, 18: 1.03125, 15: 1, 21: 1}, 'i': {18: 1.03125}, 'l': {2: 1.201378414731495, 3: 1, 7: 1.0615530303030303, 9: 1}, 'h': {26: 1.201378414731495, 28: 1, 14: 1.0909910320871516, 15: 1, 29: 1.03125}, 'f': {8: 1.1196347139148715, 7: 1.0615530303030303}, 'p': {10: 1.0909910320871516, 9: 1.03125}, 'd': {17: 1, 4: 1.3022500574996982, 15: 1}, 'm': {1: 1.0615530303030303}, 'b': {2: 1.0909910320871516, 13: 1.0909910320871516}, 'n': {29: 1.1747776359598883, 17: 1.0909910320871516, 7: 1.03125, 6: 1, 28: 1}, 'r': {14: 1.03125, 19: 1.0615530303030303, 20: 1, 5: 1, 18: 1}, 'w': {4: 1.0909910320871516}, 'c':

In [643]:
def to_probability(HistoryW: list[dict[str, dict[str, dict[int, int]]]], max_history: int):
    for h_idx in range(max_history):
        W = HistoryW[h_idx]

        for inp_symbol, s_indeces in W.items():
            symbol_stat_sum = 0

            for out_symbol, index_stat in s_indeces.items():
                for index, idx_stat in index_stat.items():
                    symbol_stat_sum += idx_stat

            for out_symbol, index_stat in s_indeces.items():
                for index, idx_stat in index_stat.items():
                    W[inp_symbol][out_symbol][index] = idx_stat / symbol_stat_sum
    
    return HistoryW
        
HistoryW = to_probability(HistoryW, max_history=max_history_L1)
RevHistoryW = to_probability(RevHistoryW, max_history=max_history_L1)

for h_idx in range(max_history_L1):
    for wk, wv in RevHistoryW[h_idx].items():
        print(wk, wv)

u0 {'o': {6: 0.5}, 'd': {0: 0.5}}
o6 {'y': {2: 0.016687532479901575, 10: 0.02146112934378399, 8: 0.017177891561037342}, 's': {5: 0.023304263857191213, 22: 0.016181849677480317, 17: 0.017177891561037342}, 't': {27: 0.021072814672950215, 16: 0.025979030588689982, 28: 0.016687532479901575, 29: 0.016181849677480317, 6: 0.016181849677480317}, 'o': {6: 0.023304263857191213, 29: 0.016687532479901575, 28: 0.016687532479901575, 17: 0.016181849677480317}, ' ': {20: 0.02810905320311339, 19: 0.018569410441233322, 22: 0.017177891561037342, 18: 0.016687532479901575, 15: 0.016181849677480317, 21: 0.016181849677480317}, 'i': {18: 0.016687532479901575}, 'l': {2: 0.019440524912954654, 3: 0.016181849677480317, 7: 0.017177891561037342, 9: 0.016181849677480317}, 'h': {26: 0.019440524912954654, 28: 0.016181849677480317, 14: 0.01765425288071339, 15: 0.016181849677480317, 29: 0.016687532479901575}, 'f': {8: 0.018117760634259128, 7: 0.017177891561037342}, 'p': {10: 0.01765425288071339, 9: 0.016687532479901575}

In [644]:
def HO_mx_diff(HO: dict[str, dict[int, int]], 
               RHO: dict[str, dict[int, int]],
               eps = 1.0):
    D: dict[str, dict[int, int]] = {}

    diff_sum = 0

    for symbol, sbl_stats in HO.items():
        rho_sbl = RHO.get(symbol, {})
        D.setdefault(symbol, {})

        for index, idx_stats in sbl_stats.items():
            rho_idx_stats = rho_sbl.get(index, 0)

            D[symbol].setdefault(index, 0)

            diff = abs(idx_stats - rho_idx_stats)

            D[symbol][index] += (rho_idx_stats - idx_stats) * eps
            diff_sum += diff

    for symbol, sbl_stats in RHO.items():
        if not HO.get(symbol):
            D.setdefault(symbol, {})

            HO.setdefault(symbol, {})

        for index, idx_stats in sbl_stats.items():
            if not HO[symbol].get(index):
                D[symbol].setdefault(index, idx_stats * eps)

                diff_sum += idx_stats
    
    return D, diff_sum



In [645]:
def get_history_stats(HistoryW, IDX, HO, inp, input_idx, h_idx):
    W = HistoryW[h_idx]
    idx = IDX[input_idx]

    O = W.get(inp + str(idx))

    for symbol, sbl_stats in O.items():
        HO.setdefault(symbol, {})

        for index, idx_stats in sbl_stats.items():
            HO[symbol].setdefault(index, 0)

            HO[symbol][index] += idx_stats

    return HO

In [646]:
def add_to_HW_update(HistoryW: list[dict[str, dict[str, dict[int, int]]]],
                     U: list[dict[str, dict[str, dict[int, int]]]],
                     IDX: list,
                     D: dict[str, dict[int, int]],
                     input: str,
                     inp_idx: int,
                     h_idx: int,
                     max_history: int, ):
    for h_idx in range(max_history):
        input_idx = inp_idx - h_idx - 1
        if input_idx < 0:
            break

        W = HistoryW[h_idx]
        idx = IDX[input_idx]

        key_I = input[input_idx] + str(idx)

        O = W.get(key_I)

        U[h_idx].setdefault(key_I, {})

        for symbol, sbl_stats in O.items():
            U[h_idx][key_I].setdefault(symbol, {})

            for index, idx_stats in sbl_stats.items():
                U[h_idx][key_I][symbol].setdefault(index, 0)

                U[h_idx][key_I][symbol][index] += idx_stats * D[symbol][index]

    return U


In [647]:
def add_to_RHW_update(RevHistoryW: list[dict[str, dict[str, dict[int, int]]]],
                      RU: list[dict[str, dict[str, dict[int, int]]]],
                      IDX: list,
                      D: dict[str, dict[int, int]],
                      input: str,
                      inp_idx: int,
                      h_idx: int,
                      max_history: int,
                      max_len: int,
                      ):
    for h_idx in range(max_history):
        input_idx = inp_idx + h_idx + 1
        if input_idx > max_len:
            break

        W = RevHistoryW[h_idx]
        idx = IDX[input_idx]

        key_I = input[input_idx] + str(idx)
        # print(f'{key_I=}')

        O = W.get(key_I)

        RU[h_idx].setdefault(key_I, {})

        for symbol, sbl_stats in O.items():
            RU[h_idx][key_I].setdefault(symbol, {})

            for index, idx_stats in sbl_stats.items():
                RU[h_idx][key_I][symbol].setdefault(index, 0)

                RU[h_idx][key_I][symbol][index] += idx_stats * -D[symbol][index]

    return RU


In [648]:
def update_HW(HW: list[dict[str, dict[str, dict[int, int]]]],
              U: list[dict[str, dict[str, dict[int, int]]]], 
              max_history: int, ):
    for h_idx in range(max_history):
        W = HW[h_idx]

        for I, O in W.items():
            if not U[h_idx].get(I):
                # print(f'Warn: U not contains: {I=}')
                break

            for symbol, sbl_stats in O.items():
                for index, idx_stats in sbl_stats.items():
                    update = U[h_idx][I][symbol][index]
                    
                    HW[h_idx][I][symbol][index] += update
    
    return HW

In [649]:
def reverse_feedback(input: str,
                     IDX: list,
                     HistoryW: list[dict[str, dict[str, dict[int, int]]]],
                     RevHistoryW: list[dict[str, dict[str, dict[int, int]]]],
                     max_history=4,
                     eps = 0.1, ):
    U: list[dict[str, dict[str, dict[int, int]]]] = []
    for hist_idx in range(max_history):
        U.append({})
    
    RU: list[dict[str, dict[str, dict[int, int]]]] = []
    for hist_idx in range(max_history):
        RU.append({})

    DT = []

    for inp_idx, inp in enumerate(input):
        if inp_idx == 0 or inp_idx > (len(input) - 1):
            continue
        # print(f'{inp_idx=}')

        HO: dict[str, dict[int, int]] = {}

        for h_idx in range(max_history):
            input_idx = inp_idx - h_idx - 1
            if input_idx < 0:
                break

            HO = get_history_stats(HistoryW, IDX, HO, input[input_idx], input_idx, h_idx)

        RHO: dict[str, dict[int, int]] = {}

        for h_idx in range(max_history):
            input_idx = inp_idx + h_idx + 1
            if input_idx > (len(input) - 1):
                break

            # print(f'{input_idx=} {h_idx=}')

            RHO = get_history_stats(RevHistoryW, IDX, RHO, input[input_idx], input_idx, h_idx)

        D, diff = HO_mx_diff(HO=HO, RHO=RHO, eps=eps)
        DT.append(diff)

        U = add_to_HW_update(HistoryW, U, IDX, D, input, inp_idx, h_idx, max_history, )
        RU = add_to_RHW_update(RevHistoryW, RU, IDX, D, input, inp_idx, h_idx, max_history, (len(input) - 1))
    
    HistoryW = update_HW(HW=HistoryW, U=U, max_history=max_history)

    RevHistoryW = update_HW(HW=RevHistoryW, U=RU, max_history=max_history)

    return HistoryW, RevHistoryW, DT

In [650]:
def plot_T(T: list):
    T_arr = np.array(T, dtype=float)

    fig, axs = plt.subplots(1, 1, figsize=(50, 5), tight_layout=True)
    axs.plot(np.arange(T_arr.size), T_arr)


    plt.show()

In [651]:
def predict(input_str: str, 
            HistoryW: list[dict[str, dict[str, dict[int, int]]]], 
            max_history = 4,
            max_index = 4,):
    err = 0
    TF = []
    AL = []

    history_indices = []
    history_indices.append([x for x in range(max_index)])

    for input_idx, input in enumerate(input_str[:-1]):
        HO: dict[str, dict[int, int]] = {}
        HO_cnt: dict[str, dict[int, int]] = {}

        itr_max_history = 0

        for h_idx in range(max_history):
            # print(f'{h_idx=}')

            inp_idx = input_idx - h_idx
            if inp_idx < 0:
                break

            itr_max_history += 1
            # print(f'{h_idx=} {inp_idx=}')
            W = HistoryW[h_idx]
            inp = input_str[inp_idx]
            indices = history_indices[h_idx]

            key_I = inp + str(indices[0])
            O = W.get(key_I, {})
            # print(f'{O=}')
            # print()

            for index in indices[1:]:
                key_I = inp + str(index)

                index_O = W.get(key_I, {})
                # print(f'{index_O=}')
                # print()

                for key, val in index_O.items():
                    O.setdefault(key, {})
                    O[key].update(val)
            
            # print(f'{O=}')
            # print()
            
            for key_O, val_O in O.items():
                HO.setdefault(key_O, {})
                HO_cnt.setdefault(key_O, {})
                
                for key_index, val_index in val_O.items():
                    HO[key_O].setdefault(key_index, 0)
                    HO_cnt[key_O].setdefault(key_index, 0)

                    # discont = (max_history - h_idx)
                    # discont = pow(0.1, h_idx)
                    discont = pow(2, (max_history - h_idx))

                    HO[key_O][key_index] += val_index * discont
                    HO_cnt[key_O][key_index] += 1

        # print(f'{HO=}')
        # print(f'{HO_cnt=}')
        # print()

        length = 0
        for v in HO.values():
            length += len(v)

        max_arr = []
        for v in HO.values():
            col_max = max(v, key=v.get)
            max_arr.append(col_max)
        
        max_arr = np.array(max_arr)
        # tf = np.mean(np.abs(max_arr - max_arr[np.newaxis].T))

        tf = length
        # tf = len(HO)

        TF.append(tf)

        # max by cnt
        # max_symbol = 'a'
        # max_symbol_index = 0
        # max_val = 0
        # for key, val in HO_cnt.items():
        #     max_idx = max(val, key=val.get)
        #     # HO_cnt[key] = {max_idx: val[max_idx]}

        #     if val[max_idx] > max_val: # 
        #         max_val = val[max_idx]
        #         max_symbol_index = max_idx
        #         max_symbol = key
        # print(f'{max_symbol=}')
        # max by val
        max_symbol = 'a'
        max_symbol_index = 0
        max_val = 0
        # win_cnt = 0
        for key, val in HO.items():
            for index, index_stat in val.items():
                index_cnt = HO_cnt[key][index]
                stat = pow(0.1, (itr_max_history - index_cnt)) * index_stat
                # stat = -0.1 * (itr_max_history - index_cnt) + index_stat

                HO[key][index] = stat

            max_idx = max(val, key=val.get)
            HO[key] = {max_idx: val[max_idx]}

            # max_cnt = HO_cnt[key][max_idx]

            if val[max_idx] > max_val: # and max_cnt >= max_his:
                # print(f'{key=} {val[max_idx]=} {max_cnt=}')
                max_val = val[max_idx]
                max_symbol_index = max_idx
                max_symbol = key
                # win_cnt = max_cnt

        # print(f'{HO=}')
        next_inp = input_str[input_idx + 1]
        # print(f'{max_symbol=}', f'{next_inp=} {max_val=} {itr_max_history=} {HO=}', max_symbol_index, max_val)
        # print('')

        # anomaly
        if max_symbol == next_inp:
            AL.append(0)
        else:
            ni_v = HO.get(next_inp, 0)
            if ni_v == 0:
                AL.append(100)
            else:
                max_ni = max(ni_v, key=ni_v.get)
                AL.append(abs(ni_v[max_ni] - max_val))

        new_indices = [max_symbol_index]

        # err
        if max_symbol != next_inp:
            err += 1

            index_stats = HO.get(next_inp, 0)
            
            if index_stats == 0:
                new_indices = [x for x in range(max_index)]
                print(f'{new_indices=}')
            else:
                max_symbol_index = list(index_stats.keys())[0]

                new_indices = [max_symbol_index]
        
        # recurtion
        history_indices.insert(0, new_indices)
        if len(history_indices) > max_history:
            history_indices = history_indices[:-1]

    TF.append(0)
    AL.append(0)

    return err, TF, AL

In [652]:
# print(f'______{IDX=}')
# print(f'{input_str=}')

err_L1, TF_L1, AL_L1 = predict(input_str, HistoryW, max_index=max_index_L1, max_history=max_history_L1)
print(err_L1, err_L1 / (len(input_str) / 100))

eps0 = 0.0001
iters = 100

for idx in range(iters):
    eps = (iters - idx) * eps0
    print(f'{eps=}')

    HistoryW, RevHistoryW, DT = reverse_feedback(input=input_str,
                                                 IDX=IDX,
                                                 HistoryW=HistoryW,
                                                 RevHistoryW=RevHistoryW,
                                                 max_history=max_history_L1,
                                                 eps=eps,)

    mean_DT = np.sum(np.array(DT))
    print(f'{mean_DT=}')
    # plot_T(T=DT)

    err_L1, TF_L1, AL_L1 = predict(
        input_str, HistoryW, max_index=max_index_L1, max_history=max_history_L1)
    print(err_L1, err_L1 / (len(input_str) / 100))


1487 29.74
eps=0.01
mean_DT=38226.977617560304
1385 27.7
eps=0.0099
mean_DT=36721.46541594339
1471 29.42
eps=0.0098
mean_DT=35593.14101793262
1521 30.42
eps=0.0097
mean_DT=34622.23506591223
1541 30.82
eps=0.009600000000000001
mean_DT=33759.87103353334
1580 31.6
eps=0.0095
mean_DT=32980.72930503161
1589 31.78
eps=0.0094
mean_DT=32269.728513415877
1607 32.14
eps=0.009300000000000001
mean_DT=31616.17155974319
1616 32.32
eps=0.0092
mean_DT=31011.115109253944
1639 32.78
eps=0.0091
mean_DT=30448.096229435534
1636 32.72
eps=0.009000000000000001
mean_DT=29922.308561943457
1640 32.8
eps=0.0089
mean_DT=29429.55931855282
1649 32.98
eps=0.0088
mean_DT=28966.13949203192
1652 33.04
eps=0.008700000000000001
mean_DT=28528.959531009445
1656 33.12
eps=0.0086
mean_DT=28115.508699337952
1661 33.22
eps=0.0085
mean_DT=27723.58605800676
1666 33.32
eps=0.008400000000000001
mean_DT=27351.400138289748
1671 33.42
eps=0.0083
mean_DT=26997.363907625877
1672 33.44
eps=0.0082
mean_DT=26659.958611276288
1680 33.6
eps

- Проверить без нормализации
- Проверить с нормализацией после каждого апдейта

In [653]:
def get_word_stats(input_str: str | list, TF: list, threshold=0.0):
    buffer: list = [input_str[0]]
    WordStats: dict[str, int] = {}
    WordIndex: dict[str, int] = {}
    next_wrd_idx = 0

    NextLayerOut = []
    DownstreamIndex = {}

    prev_tf = TF[0]

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 and inp_idx >= len(input_str) - 1:
            continue

        # if TF[inp_idx] - prev_tf > 0.0: # and abs(TF[inp_idx] - prev_tf) > 0.2:
        # and TF[inp_idx] > threshold:
        if TF[inp_idx] > prev_tf and TF[inp_idx] > TF[inp_idx + 1]:
            word = ''.join(buffer)

            WordStats.setdefault(word, 0)
            WordStats[word] += 1

            if not WordIndex.get(word):
                WordIndex.setdefault(word, next_wrd_idx)
                next_wrd_idx += 1

            wrd_idx = WordIndex.get(word)
            # NextLayerOut.append(str(wrd_idx))
            NextLayerOut.append(word)

            DownstreamIndex.setdefault(inp_idx, 1)

            buffer = [inp]
            prev_tf = TF[inp_idx]

            continue

        TF[inp_idx] = 0

        buffer.append(inp)
        prev_tf = TF[inp_idx]

    # sort
    WordStats = dict(sorted(WordStats.items(), key=lambda item: item[1], reverse=True))

    NextLayerOut.append(str(next_wrd_idx))
    return WordStats, WordIndex, NextLayerOut, DownstreamIndex


TF_L1 = np.array(TF_L1, dtype='float64')
AL_L1 = np.array(AL_L1, dtype='float64')
TF_L1 /= np.max(TF_L1)
AL_L1 /= np.max(AL_L1)
TF_L1 += AL_L1

TF_max_coef = 0.4

# TF_L1[np.where(TF_L1 < np.mean(TF_L1))] = 0
# TF_1 = TF_L1 - np.pad(TF_L1, (1, 0))[:-1]
# TF_2 = TF_L1 - np.pad(TF_L1, (0, 1))[1:]
# TF_L1 = TF_1 + TF_2

# TF_L1[np.where(TF_L1 < 0)] = 0
# TF_L1[np.where(TF_L1 < TF_max_coef)] = 0
TF_L1 = TF_L1.tolist()

WordStats_L1, WordIndex_L1, NextLayerOut_L1, DownstreamIndex_L1 = get_word_stats(input_str=input_str,
                                                                                 TF=TF_L1, 
                                                                                 threshold=0.0, )
print(len(WordStats_L1))
mean_WS = np.mean(np.array(list(WordStats_L1.values())))
print(f'{mean_WS=}')
WS_len = 0
for key, val in WordStats_L1.items():
    # if len(key.strip()) > 1 and val >= mean_WS:
    if val >= mean_WS:
        WS_len += 1
    print(key, val)
print(f'{WS_len=}')

print(f'{NextLayerOut_L1=}')
print(f'{len(NextLayerOut_L1)=}')

print(f'{WordIndex_L1=}')

print(f'{DownstreamIndex_L1=}')


747
mean_WS=2.4497991967871484
 a 39
 b 36
 h 30
 t 29
th 28
 s 21
 w 20
e  20
 c 19
ng 19
st 18
 i 17
id 16
he 15
on 15
t  15
as 15
en 14
nd 14
in 12
ou 12
he  11
 y 11
sa 11
 l 11
 th 10
at 10
of 10
 he 10
 to 10
is 9
 m 9
an 9
nd  9
 f 9
er 9
do 8
ea 8
oh 8
or 8
ed  8
ng  7
 r 7
 p 7
 g 7
g  7
 d 7
s  7
 he  7
it 7
aid 7
ee 7
 no 7
the 7
ad 7
ere 6
ow 6
d  6
bin 6
es  6
 po 6
es 6
ime 6
hri 6
op 6
the  6
uzzi 6
at  6
 be 6
el 6
and 6
ed 5
 n 5
om 5
o  5
 wi 5
 sa 5
 do 5
be 5
ik 5
rs 5
ly  5
y  5
r ro 5
 i  5
of  5
 it 5
all 5
as  4
ly 4
hat 4
tr 4
ooh 4
 j 4
re 4
al 4
pl 4
et 4
 so 4
ns 4
 li 4
is  4
so 4
ll 4
de 4
wh 4
 to  4
 o 4
nk 4
if 4
ch 4
oph 4
er r 4
ob 4
we 3
ie  3
ome 3
wa 3
ther 3
od 3
nn 3
 wh 3
ie 3
 yo 3
ha 3
ns  3
no 3
i  3
 q 3
ga 3
ni 3
one  3
elf 3
 v 3
ong 3
out  3
r  3
 in  3
gr 3
on  3
ing 3
le 3
t t 3
ee  3
hi 3
 you  3
n' 3
aki 3
for 3
ei 3
ey  3
n  3
tt 3
and  3
ust 3
gh 3
 we 3
ou  3
nt 3
ri 3
oo 3
omi 2
bu 2
hr 2
her  2
ar  2
 of 2
 but 2
uld 2
 of  2
 is